# Importance sampling

In [4]:
prefix = '/home/ines/repositories/'
prefix = '/Users/ineslaranjeira/Documents/Repositories/'

In [5]:

""" 
IMPORTS
"""
import os
import numpy as np
import pickle
import pandas as pd
from one.api import ONE
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from scipy import stats

# Get my functions
functions_path =  prefix + 'representation_learning_variability/Models/Sub-trial//2_fit_models/'
os.chdir(functions_path)
from preprocessing_functions import idxs_from_files
functions_path =  prefix + 'representation_learning_variability/Functions/'
os.chdir(functions_path)
from data_processing import save_and_log

one = ONE(mode='remote')

In [6]:
# LOAD DATA
bin_size = 0.017

num_train_batches = 5

data_path =prefix + 'representation_learning_variability/DATA/Sub-trial/Design matrix/' + 'v6_21Jul2025/' + str(bin_size) + '/'
all_files = os.listdir(data_path)
design_matrices = [item for item in all_files if 'design_matrix' in item and 'standardized' not in item]
idxs, mouse_names = idxs_from_files(design_matrices, bin_size)

# Subsample and embedd 

In [7]:
# Identify sessions availableto process
sessions_to_process = []
for m, mat in enumerate(idxs):
    mouse_name = mat[37:]
    session = mat[:36]
    sessions_to_process.append((mouse_name, session))

In [10]:
def plot_kde(X_embedded, kernel):
    xmin = -150
    xmax = 150
    ymin=-150
    ymax=150
    X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([X.ravel(), Y.ravel()])
    Z = np.reshape(kernel(positions).T, X.shape)

    fig, ax = plt.subplots()
    ax.imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r,
            extent=[xmin, xmax, ymin, ymax])
    ax.plot(X_embedded[:, 0], X_embedded[:, 1], 'k.', markersize=2)
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    plt.show()

In [16]:
concatenated_subsampled = np.array([])

var_interest = ['0.5', '1.0', '2.0', '4.0', '8.0']

for m, mat in enumerate(sessions_to_process[:100]):

    mouse_name = mat[0]
    session = mat[1]
    fit_id = str(mouse_name+session)
    filename = data_path + "design_matrix_" + str(session) + '_'  + mouse_name
    original_design_matrix = pd.read_parquet(filename)
    array_matrix = np.array(original_design_matrix)

    # Remove NaNs
    filtered_matrix = array_matrix[~np.isnan(array_matrix).any(axis=1)]
    keys = np.where(original_design_matrix.keys().isin(var_interest))
    design_matrix = filtered_matrix[:, keys]
    design_matrix = np.reshape(design_matrix,(np.shape(design_matrix)[0], np.shape(design_matrix)[2]))

    # Prepare data
    num_timesteps = np.shape(design_matrix)[0]
    data = design_matrix[:(num_timesteps // num_train_batches) * num_train_batches]
    
    """ Randomly subsample """
    n_samples = 20000
    sampled_indices = np.random.choice(data.shape[0], 
                                       n_samples, 
                                       replace=False)
    sampled_data = data[sampled_indices, :]
    
    """ Perform t-SNE """
    X = sampled_data.copy()
    X_embedded = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=32).fit_transform(X)
    # plt.scatter(X_embedded[:, 0], X_embedded[:, 1], alpha=0.02)
    
    """ Perform Gaussian kde """
    values = X_embedded.T.copy()
    kernel = stats.gaussian_kde(values)
    # plot_kde(X_embedded, kernel)
    
    """ Resample weighted by kde estimate """
    sample_prob = kernel(values)
    norm_sample_prob = sample_prob/ np.sum(sample_prob)
    resampled_indices = np.random.choice(sampled_data.shape[0], 
                                    size=2000, 
                                    p=norm_sample_prob, replace=False)
    resampled_data = sampled_data[resampled_indices, :]
    
    """ Concatenate session """
    if len(concatenated_subsampled) == 0:
        concatenated_subsampled = resampled_data.copy()
    else:
        concatenated_subsampled = np.vstack([concatenated_subsampled, resampled_data])
    
    print(mat)

('NR_0027', '0f77ca5d-73c2-45bd-aa4c-4c5ed275dbde')
('NYU-47', '288bfbf3-3700-4abe-b6e4-130b5c541e61')
('UCLA006', '19e66dc9-bf9f-430b-9d6a-acfa85de6fb7')
('CSH_ZAD_026', '5d01d14e-aced-4465-8f8e-9a1c674f62ec')
('NYU-65', '8c025071-c4f3-426c-9aed-f149e8f75b7b')
('UCLA036', 'e2b845a1-e313-4a08-bc61-a5f662ed295e')
('NR_0020', 'eacc49a9-f3a1-49f1-b87f-0972f90ee837')
('UCLA033', '32d27583-56aa-4510-bc03-669036edad20')
('UCLA017', 'bb099402-fb31-4cfd-824e-1c97530a0875')
('UCLA048', 'dc962048-89bb-4e6a-96a9-b062a2be1426')
('NYU-45', '537677fe-1e24-4755-948c-fa4a4e8ecce5')
('UCLA034', '8a3a0197-b40a-449f-be55-c00b23253bbf')
('NYU-21', 'ebe2efe3-e8a1-451a-8947-76ef42427cc9')
('ibl_witten_20', '037d75ca-c90a-43f2-aca6-e86611916779')
('NYU-48', 'a405053a-eb13-4aa4-850c-5a337e5dc7fd')
('NYU-48', '3d59aa1a-b4ba-47fe-b9cf-741b5fdb0c7b')
('UCLA049', '8a1cf4ef-06e3-4c72-9bc7-e1baa189841b')
('ibl_witten_29', 'af55d16f-0e31-4073-bdb5-26da54914aa2')
('UCLA049', '0841d188-8ef2-4f20-9828-76a94d5343a4')
('

KeyboardInterrupt: 

## Save data

In [65]:
assert concatenated_subsampled.shape[0] == 2000 * len(sessions_to_process)

In [19]:
np.save(open('supersession_wavelets08-20-2025', 'wb'), concatenated_subsampled)

In [ ]:
file_to_save = concatenated_subsampled.copy()
filename = 'supersession_wavelets'
save_path = prefix + '/representation_learning_variability/Models/Maths intuitions/Wavelets/'
file_format = 'npy'
script_name = 'importance_sampling.ipynb'
metadata = save_and_log(file_to_save, filename, file_format, save_path, script_name)